In [1]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../..")

import infrasap.market_access as ma
import infrasap.rasterMisc as rMisc

In [2]:
global_friction_file    = "/home/public/Data/GLOBAL/INFRA/FRICTION_2015/2015_friction_surface_v1.geotiff"
global_friction_noOcean = "/home/public/Data/GLOBAL/INFRA/FRICTION_2015/2015_friction_surface_v1_no_ocean_travel.tif"
ports_file = "/home/wb411133/data/Global/INFRA/PORTS/major_ports.shp"



In [ ]:
if not os.path.exists(global_friction_noOcean):# Burn the oceans in as a no travel zone
    inG = rasterio.open(global_friction_file)
    inG_data = inG.read()[0,:,:]
    inG_data = inG_data * 1000
    
    ocean_file = "/home/public/Data/GLOBAL/ADMIN/OCEAN/ne_10m_ocean.shp"
    cMeta = inG.meta.copy()

    inO = gpd.read_file(ocean_file)
    shapes = ((row.geometry,999999) for idx, row in inO.iterrows())
    burned = features.rasterize(shapes=shapes, out_shape=(cMeta['height'], cMeta['width']), 
                                transform=cMeta['transform'], dtype=cMeta['dtype'])
    inG_combo = inG_data + burned
    out_file = "/home/wb411133/temp/2015_friction_surface_v1_no_ocean_travel.tif"
    with rasterio.open(out_file, 'w', **cMeta) as out:
        out.write_band(1, inG_combo)
   

In [61]:
inG = rasterio.open(global_friction_noOcean)
inP = gpd.read_file(ports_file)

inG_data = inG.read()[0,:,:]
mcp = graph.MCP_Geometric(inG_data)

In [5]:
# Attach ISO3 codes and regions to ports
global_boundaries =  "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
inB = gpd.read_file(global_boundaries)

In [34]:
inP = gpd.read_file(ports_file)
if inP.crs != inB.crs:
    inP = inP.to_crs(inB.crs)
    
inP['ISO3'] = ''
inP['REGION'] = ''
for idx, row in inP.iterrows():
    sel_country = inB.loc[inB.intersects(row['geometry'])]
    if sel_country.shape[0] > 0:
        inP.loc[idx,'ISO3'] = sel_country['ISO3'].iloc[0]
        inP.loc[idx,'REGION'] = sel_country['Region'].iloc[0]
    else:
        print(f"{idx}: {row['Postal']}")
              
inP = inP.to_crs({'init':'epsg:4326'})
inP.to_file(ports_file)

In [65]:
selP = inP#.loc[inP['REGION'] == 'South Asia']

In [66]:
travel_time_thresholds = [((24*60) * x) for x in [0.25,0.5,1,2]]
out_folder = os.path.join(os.path.dirname(ports_file), "TRAVEL_TIMES")
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
port_files = []
for idx, row in selP.iterrows():
    name = row['OBJECTID']    
    out_file = os.path.join(out_folder, "PORT_ISOCHRONES_NO_OCEAN_%s.shp" % name)
    port_files.append(out_file)
    if not os.path.exists(out_file):
        print("Processing %s" % name)
        current_p = pd.DataFrame(selP.loc[idx,]).transpose()
        travel_times = ma.generate_feature_vectors(inG, mcp, current_p, travel_time_thresholds)
        travel_times.to_file(out_file)
        
    

Processing 19
13:13:37	1 of 1
Processing 20
13:26:37	1 of 1
Processing 23
13:39:31	1 of 1
Processing 24
13:51:56	1 of 1
Processing 41
14:04:20	1 of 1
Processing 46
14:17:02	1 of 1
Processing 47
14:29:34	1 of 1
Processing 51
14:42:15	1 of 1
Processing 63
14:54:49	1 of 1
Processing 89
15:07:40	1 of 1
Processing 122
15:20:55	1 of 1
Processing 124
15:33:36	1 of 1
Processing 132
15:47:49	1 of 1
Processing 138
16:00:20	1 of 1
Processing 152
16:13:23	1 of 1
Processing 165
16:25:43	1 of 1
Processing 166
16:39:35	1 of 1
Processing 173
16:53:34	1 of 1
Processing 175
17:06:07	1 of 1
Processing 103
17:19:51	1 of 1
Processing 102
17:33:07	1 of 1
Processing 104
17:46:11	1 of 1
Processing 105
17:59:34	1 of 1
Processing 55
18:12:48	1 of 1
Processing 62
18:25:39	1 of 1
Processing 61
18:38:27	1 of 1
Processing 67
18:51:16	1 of 1
Processing 114
19:03:52	1 of 1
Processing 45
19:16:24	1 of 1
Processing 44
19:28:54	1 of 1
Processing 127
19:41:41	1 of 1
Processing 126
19:55:00	1 of 1
Processing 119
20:08:22	

In [68]:
try:
    del final
except:
    pass
for p in port_files:
    current_p = gpd.read_file(p)
    current_p['PORT'] = p.split("_")[-1].replace(".shp","")
    try:
        final = final.append(current_p)
    except:
        final = current_p

In [69]:
final.to_file(os.path.join(out_folder, "COMBO_all_ports.shp"))

In [70]:
out_folder

'/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES'

# Zonal Stats

In [67]:
inShape = os.path.join(out_folder, "COMBO_all_ports.shp")